In [ ]:
from pathlib import Path
import numpy as np
import importlib
from matplotlib import pyplot as plt
import humanize
import sys
import os, psutil
import time
import shutil

import astrocast.reduction as red
import astrocast.clustering as clust
import astrocast.analysis as ana
import astrocast.autoencoders as AE
import astrocast.helper as helper

for pack in [red, clust, helper, ana, AE, astroHelper]:
    importlib.reload(pack)


In [ ]:
# default settings signal generator
def_gen = dict(noise_amplitude=0.0001, trace_length=(50, 50))
def_dummy = dict(num_rows=1000, )


# Generate example datasets

In [ ]:
# Identical
sig_1 = helper.SignalGenerator(**def_gen)
sig_2 = helper.SignalGenerator(**def_gen)

dg = helper.DummyGenerator(generators=(sig_1, sig_2), **def_dummy)
e_ident = dg.get_events()
# display(e_ident)
e_ident.name = "ident"

plot_ident = ana.Plotting(e_ident)
ax = plot_ident.plot_traces(num_samples=4, by="group", figsize=(2, 2))

In [ ]:
# different
sig_1 = helper.SignalGenerator(b=2, plateau_duration=6, **def_gen)
sig_2 = helper.SignalGenerator(**def_gen)

dg = helper.DummyGenerator(generators=(sig_1, sig_2), **def_dummy)
e_diff = dg.get_events()
# display(e_diff)
e_diff.name = "diff"

plot_diff = ana.Plotting(e_diff)
ax = plot_diff.plot_traces(num_samples=4, by="group", figsize=(2, 2))

# Conditional Constrasts

## Reduction

In [ ]:
embeddings = {}

### Feature Extraction

In [ ]:
feature_extraction = {}

for eObj in [e_ident, e_diff]:
    
    fe = red.FeatureExtraction(eObj)
    features = fe.all_features(dropna=True)
    feature_extraction[eObj.name] = features.values.astype(float)

embeddings["fe"] = feature_extraction

### CNN Autoencoder

In [ ]:
cnn_autoencoder = {}

for eObj in [e_ident, e_diff]:
    
    target_length = def_gen["trace_length"][0]
    data = np.array(eObj.events.trace.tolist())
    display(data.shape)
    
    cnn = AE.CNN_Autoencoder(target_length=target_length, use_cuda=True)
    X_train, X_val, X_test = cnn.split_dataset(data=data)
    
    cnn.train_autoencoder(X_train=X_train, X_val=X_val, epochs=25)
    
    embedding = cnn.embed(data)
    cnn_autoencoder[eObj.name] = embedding.astype(float)

embeddings["cnn"] = cnn_autoencoder




### RNN Autoencoder

In [ ]:
rnn_autoencoder = {}

for eObj in [e_ident, e_diff]:
    
    pdl = AE.PaddedDataLoader(data=eObj.events.trace)
    X_train, X_val, X_test = pdl.get_datasets(batch_size=16,
                                              val_size=0.1,
                                              test_size=0.05)
    
    tRAE = AE.TimeSeriesRnnAE(use_cuda=True)
    tRAE.train_epochs(dataloader_train=X_train,
                      dataloader_val=X_val,
                      num_epochs=10,
                      patience=10,
                      safe_after_epoch=None,
                      show_mode='progress'
                      )
    
    X = pdl.get_dataloader(data=eObj.events.trace, batch_size=16, shuffle=False)
    _, _, embedding, _ = tRAE.embedd(X)
    
    rnn_autoencoder[eObj.name] = np.array(embedding).astype(float)

embeddings["rnn"] = rnn_autoencoder

## Classifier (Predict condition)

In [ ]:
importlib.reload(clust)

for eObj in [e_ident, e_diff]:
    for emb_name, embedding in embeddings.items():
        discr = clust.Discriminator(eObj)
        
        clf = discr.train_classifier(embedding=embedding[eObj.name], category_vector=eObj.events.group.tolist())
        discr.evaluate(show_plot=True, title=f"condition {i}: {eObj.name} [{emb_name}]", figsize=(8, 4))
